In [24]:
# constant variable

T5_ORI_LEN = 32100

TAG_TO_WORD = {'POS': 'positive', 'NEG': 'negative', 'NEU': 'neutral'}
TAG_WORD_LIST = ['positive', 'negative', 'neutral']

# annotation-bracket
TAG_TO_BRACKET = {"POS": ("((", "))"), "NEG": ("[[", "]]"), "NEU": ("{{", "}}")}
BRACKET_TO_TAG = {"{{": "neutral", "[[": "negative", "((": "positive"}

# annotation-special
NONE_TOKEN = "[none]"
AND_TOKEN = "[and]"
ASPECT_TOKEN = "<aspect>"
OPINION_TOKEN = "<opinion>"
EMPTY_TOKEN = "<empty>"
SEP_TOKEN = "<sep>"
TAG_TO_SPECIAL = {"POS": ("<pos>", "</pos>"), "NEG": ("<neg>", "</neg>"), "NEU": ("<neu>", "</neu>")}
SPECIAL_TO_TAG = {"<pos>": "positive", "<neg>": "negative", "<neu>": "neutral"}

TARGET_TEST_COUNT_DICT = {
    "laptop": 800,
    "rest": 2158,
    "device": 1279,
    "service": 747,
}
# please follw L, R, D, S order for evaluation purpose, since we will combine test files into one

ASTE_TRANSFER_PAIRS = {
    "rest14": ["laptop14"],
    # "rest15": ["laptop14"],
    # "rest16": ["laptop14"],
    # "laptop14": ["rest14", "rest15", "rest16"],
}
AOPE_TRANSFER_PAIRS = ASTE_TRANSFER_PAIRS

STOP_WORDS = ['about', 'itself', 'so', 'further', 'against', "don't", 'shouldn', 'to', 'didn', 'hers', 'over', 'haven', "it's", 'of', 'have', 'm', 'but', "you've", 'which', 'd', 'most', 'nor', "haven't", "wasn't", 'yourself', 'with', 'am', 'do', 'than', "that'll", "isn't", 'or', "shan't", 'then', 'while', 'did', 'off', 'under', "mustn't", "won't", 'again', 'you', 'its', 'these', 'some', 'he', 'after', 'doesn', 'into', 't', 'more', 'whom', 'his', 'from', 'a', 'at', 'during', 'when', "she's", "aren't", 'was', 'same', 'myself', 'my', 'has', 'aren', 'by', 'before', "needn't", 'yourselves', 'such', 'she', 'is', 'needn', 'here', 'too', 'ourselves', "didn't", 'both', 'i', 'theirs', 'weren', 'be', 'their', 'were', 'because', 'should', "should've", "couldn't", 'will', 'isn', 'all', 'and', 'through', 'won', "weren't", 'y', 'they', 'for', 'until', 'him', 's', 'now', 'those', 'up', 'had', 'that', 'ma', 'couldn', 'been', 'why', 'below', 'own', 'doing', "you'll", 'very', 'above', "shouldn't", 'where', 've', 'if', 'are', 'how', 'wasn', 'it', 'what', 'as', 'hadn', 'hasn', "you'd", "wouldn't", 'don', 'few', 'other', 're', 'ain', "hadn't", "doesn't", 'himself', 'shan', 'the', 'not', 'mustn', 'does', "hasn't", 'll', 'your', 'yours', 'herself', 'in', 'wouldn', 'themselves', 'who', 'there', 'ours', 'out', 'mightn', 'me', 'them', 'once', "mightn't", 'we', 'her', 'this', 'being', 'any', 'can', 'o', 'no', 'having', "you're", 'our', 'on', 'between', 'down', 'only', 'just', 'each', 'an']

In [3]:
test = "MS Office 2011 for Mac is wonderful , well worth it .####[([0, 1, 2, 3, 4], [6], 'POS'), ([0, 1, 2, 3, 4], [8, 9], 'POS')]"
test_processed = "MS Office 2011 for Mac is wonderful , well worth it . ===> <pos> MS Office 2011 for Mac <opinion> wonderful <pos> MS Office 2011 for Mac <opinion> well worth"

In [6]:
def read_line_examples_from_file(data_path):
    """
    Read data from file, each line is: sent####labels
    Return List[List[word]], List[Tuple]
    """
    sents, labels = [], []
    with open(data_path, 'r', encoding='UTF-8') as fp:
        words, labels = [], []
        for line in fp:
            line = line.strip()
            if line != '':
                words, tuples = line.split('####')
                sents.append(words.split())
                labels.append(eval(tuples))
    print(f"{data_path.split('/')[-1]}\tTotal examples = {len(sents)} ")
    return sents, labels

def prepare_aste_universal(args, data_type_file="train"):
    if data_type_file:
        data_path = f"{args.data_dir}/{data_type_file}.txt"
        sents, labels = read_line_examples_from_file(data_path)
    
    inputs = [" ".join(s) for s in sents]
    targets = []
    for i, label in enumerate(labels):
        all_tri = []
        target_str = ""
        for tri in label:
            if len(tri[0]) == 1:
                aspect = sents[i][tri[0][0]]
            else:
                start_idx, end_idx = tri[0][0], tri[0][-1]
                aspect = ' '.join(sents[i][start_idx:end_idx+1])
            if len(tri[1]) == 1:
                opinion = sents[i][tri[1][0]]
            else:
                start_idx, end_idx = tri[1][0], tri[1][-1]
                opinion = ' '.join(sents[i][start_idx:end_idx+1])
            senti_token = TAG_TO_SPECIAL[tri[2]][0]
            target_str += f" {senti_token} {aspect} {OPINION_TOKEN} {opinion}"
        targets.append(target_str.strip())
    return inputs, targets

In [7]:
import os
os.getcwd()

'c:\\Workspace\\Tutorial\\BGCA'

In [22]:
sents, labels = read_line_examples_from_file('outputs/aste/cross_domain/1024_1313-run_aste/seed-42/rest14-laptop14/data/test.txt')
print(f'sents (tokenize) --> {sents[:5]}')
print(f'labels --> {labels[:5]}')
print('>>------->---<-------<<')

inputs = [" ".join(s) for s in sents]
print(f'(list of sent (text)) inputs --> {inputs[:5]}')

test.txt	Total examples = 100 
sents (tokenize) --> [['Boot', 'time', 'is', 'super', 'fast', ',', 'around', 'anywhere', 'from', '35', 'seconds', 'to', '1', 'minute', '.'], ['tech', 'support', 'would', 'not', 'fix', 'the', 'problem', 'unless', 'I', 'bought', 'your', 'plan', 'for', '$', '150', 'plus', '.'], ['Set', 'up', 'was', 'easy', '.'], ['Did', 'not', 'enjoy', 'the', 'new', 'Windows', '8', 'and', 'touchscreen', 'functions', '.'], ['Other', 'than', 'not', 'being', 'a', 'fan', 'of', 'click', 'pads', '(', 'industry', 'standard', 'these', 'days', ')', 'and', 'the', 'lousy', 'internal', 'speakers', ',', 'it', "'s", 'hard', 'for', 'me', 'to', 'find', 'things', 'about', 'this', 'notebook', 'I', 'do', "n't", 'like', ',', 'especially', 'considering', 'the', '$', '350', 'price', 'tag', '.']]
labels --> [[([0, 1], [4], 'POS')], [([0, 1], [3, 4], 'NEG')], [([0, 1], [3], 'POS')], [([5, 6], [1, 2], 'NEG'), ([8, 9], [1, 2], 'NEG')], [([18, 19], [17], 'NEG')]]
>>------->---<-------<<
(list of sent 

In [26]:
targets = []
for i, label in enumerate(labels):
        print(f'============={i}============')
        all_tri = []
        target_str = ""
        for tri in label:
            print(tri)   # ([list of aspect word], [list of opinion word], SENTIMENT)

            ## aspect words
            if len(tri[0]) == 1: 
                aspect = sents[i][tri[0][0]]
            else:
                start_idx, end_idx = tri[0][0], tri[0][-1]
                aspect = ' '.join(sents[i][start_idx:end_idx+1])

            ## opinion words
            if len(tri[1]) == 1: 
                opinion = sents[i][tri[1][0]]
            else:
                start_idx, end_idx = tri[1][0], tri[1][-1]
                opinion = ' '.join(sents[i][start_idx:end_idx+1])

            ## sentiment    
            senti_token = TAG_TO_SPECIAL[tri[2]][0]


            target_str += f" {senti_token} {aspect} {OPINION_TOKEN} {opinion}"
            print(f'{target_str}')
      
        targets.append(target_str.strip())

=============0============
([0, 1], [4], 'POS')
 <pos> Boot time <opinion> fast
=============1============
([0, 1], [3, 4], 'NEG')
 <neg> tech support <opinion> not fix
=============2============
([0, 1], [3], 'POS')
 <pos> Set up <opinion> easy
=============3============
([5, 6], [1, 2], 'NEG')
 <neg> Windows 8 <opinion> not enjoy
([8, 9], [1, 2], 'NEG')
 <neg> Windows 8 <opinion> not enjoy <neg> touchscreen functions <opinion> not enjoy
=============4============
([18, 19], [17], 'NEG')
 <neg> internal speakers <opinion> lousy
=============5============
([9], [2], 'POS')
 <pos> use <opinion> fast
([9], [4], 'POS')
 <pos> use <opinion> fast <pos> use <opinion> light
([9], [7], 'POS')
 <pos> use <opinion> fast <pos> use <opinion> light <pos> use <opinion> simple
=============6============
([0], [1], 'POS')
 <pos> Works <opinion> well
([13, 14], [7], 'POS')
 <pos> Works <opinion> well <pos> apple OS <opinion> happy
=============7============
([9], [3, 4, 5, 6], 'POS')
 <pos> features <o